# * Ex-D Data

## Parameter

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [3]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-05-09, 17:06:35

   -> Summany DataFrame : 125 rows, 20 columns

TDMDBPR : Disconnected


In [4]:
active_df = chk_src_df.loc[chk_src_df['KPI_FLAG']=='Active'].copy()
active_df = active_df.fillna(0).sort_values(by=['KPI_FLAG', 'BASENAME', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = active_df.iloc[:, 9:17].columns.tolist()
for col in mod_col_list:
    active_df[col] = active_df[col].apply(lambda x: format(x, ',.0f'))

# active_df
print(f'\n{active_df.to_string(max_cols=100)}')


   KPI_FLAG   PAR_KEY                                BASENAME    METRIC_CD                                           METRIC_NAME CHANNEL_CD               LOAD_DATE  START_DAY   END_DAY              P              G               H              HH CCAA        H_080 H_016 H_040  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0    Active  20250506                                       0   DB2S000201                  Postpaid Churn Subs Voluntary : DTAC        ALL 2025-05-08 19:19:32.000   20250427  20250506          3,542          7,084           7,075           7,075    0          189     0     0              4          170     1639
1    Active  20250506                                       0   DB2S000202                Postpaid Churn Subs Involuntary : DTAC        ALL 2025-05-08 19:19:33.000   20250427  20250506             80            160             160             160    0            2     0     0              4          170     1639
2    Active  20250506    D_CORPKPI_ACTUAL_01_20250506.txt

In [5]:
obsolete_df = chk_src_df.loc[chk_src_df['KPI_FLAG']=='Obsolete'].copy()
obsolete_df = obsolete_df.loc[obsolete_df['CHANNEL_CD']=='ALL']
obsolete_df = obsolete_df.fillna(0).sort_values(by=['KPI_FLAG', 'BASENAME', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = obsolete_df.iloc[:, 9:17].columns.tolist()
for col in mod_col_list:
    obsolete_df[col] = obsolete_df[col].apply(lambda x: format(x, ',.0f'))

# obsolete_df
print(f'\n{obsolete_df.to_string(max_cols=100)}')


   KPI_FLAG   PAR_KEY                              BASENAME     METRIC_CD                      METRIC_NAME CHANNEL_CD  LOAD_DATE  START_DAY   END_DAY           P            G            H           HH CCAA      H_080 H_016 H_040  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0  Obsolete  20250506  D_CORPKPI_ACTUAL_01_20250506.txt.gpg  DB0R00010001           Total Inflow M1 : DTAC        ALL 2025-05-06   20250427  20250506  59,369,416  118,738,831  237,318,615  118,596,339    0  2,195,150     0     0              4          170     1890
1  Obsolete  20250506  D_CORPKPI_ACTUAL_01_20250506.txt.gpg    DB1R000900         Prepaid Inflow M1 : DTAC        ALL 2025-05-06   20250427  20250506  42,064,225   84,128,450   83,989,163   83,989,163    0    895,456     0     0              4          170     1890
2  Obsolete  20250506  D_CORPKPI_ACTUAL_01_20250506.txt.gpg    DB1R001000         Prepaid Inflow M2 : DTAC        ALL 2025-05-06   20250427  20250506  20,870,567   41,741,134   41,782,232   41,782,232 

In [6]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250506,DB1R000100,Prepay Revenue,2025-05-06,20250427,20250506,"627,257,927","1,254,515,854","1,253,424,111","1,253,424,111",0,"13,672,182",4,170,1850
1,20250506,DB2R000100,Postpaid Revenue,2025-05-06,20250427,20250506,"1,054,778,532","2,109,557,064","2,107,488,217","2,107,488,217",0,"37,701,204",4,170,1836
2,20250506,DB2R010100,Postpaid Revenue B2C,2025-05-06,20250427,20250506,"893,368,883","1,786,737,766","1,785,203,187","1,785,203,187",0,"32,538,086",4,170,1832
3,20250506,DB2R020100,Postpaid Revenue B2B,2025-05-06,20250427,20250506,"161,409,649","322,819,297","322,285,030","322,285,030",0,"5,163,118",4,170,1826


In [7]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250506,DB1S000500,Prepay Usage Subs,2025-05-06,20250427,20250506,"112,869,302","225,738,604","203,047,191","203,047,191",0,"2,273,957",4,170,1850
1,20250506,DB2S000500,Postpaid Active Subs,2025-05-06,20250427,20250506,"57,939,908","115,879,816","104,204,115","104,204,115",0,"1,732,946",4,170,1850
2,20250506,DB2S010500,Postpaid Active Subs B2C,2025-05-06,20250427,20250506,"51,065,280","102,130,560","91,848,503","91,848,503",0,"1,560,255",4,170,1850
3,20250506,DB2S020400,Postpaid Active Sub B2B,2025-05-06,20250427,20250506,"6,874,628","13,749,256","12,355,612","12,355,612",0,"172,691",4,170,1729


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [8]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-05-09, 17:06:36

   -> Daily DataFrame : 80 rows, 16 columns

TDMDBPR : Disconnected


In [9]:
''' DB1R000100 : Prepay Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20250501


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250506,DB1R000100,Prepay Revenue,20250501,2025-05-06,"75,051,850","150,103,701","149,966,955","149,966,955",0,"1,635,728",0,0,4,170,185
1,20250506,DB1R000100,Prepay Revenue,20250502,2025-05-06,"68,739,462","137,478,925","137,361,262","137,361,262",0,"1,530,001",0,0,4,170,185
2,20250506,DB1R000100,Prepay Revenue,20250503,2025-05-06,"62,101,701","124,203,402","124,103,324","124,103,324",0,"1,401,946",0,0,4,170,185
3,20250506,DB1R000100,Prepay Revenue,20250504,2025-05-06,"59,280,894","118,561,788","118,458,386","118,458,386",0,"1,315,014",0,0,4,170,185
4,20250506,DB1R000100,Prepay Revenue,20250505,2025-05-06,"60,823,569","121,647,139","121,540,538","121,540,538",0,"1,314,119",0,0,4,170,185
5,20250506,DB1R000100,Prepay Revenue,20250506,2025-05-06,"60,515,920","121,031,839","120,928,164","120,928,164",0,"1,271,558",0,0,4,170,185


In [10]:
''' DB2R000100 : Postpaid Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20250501


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250506,DB2R000100,Postpaid Revenue,20250501,2025-05-06,"9,957,565","19,915,130","19,904,033","19,904,033",0,"313,936",0,0,4,170,183
1,20250506,DB2R000100,Postpaid Revenue,20250502,2025-05-06,"231,461,029","462,922,058","462,204,410","462,204,410",0,"8,788,337",0,0,4,170,183
2,20250506,DB2R000100,Postpaid Revenue,20250503,2025-05-06,"90,325,865","180,651,729","180,511,900","180,511,900",0,"3,176,917",0,0,4,170,183
3,20250506,DB2R000100,Postpaid Revenue,20250504,2025-05-06,"88,400,498","176,800,995","176,662,014","176,662,014",0,"3,115,817",0,0,4,170,183
4,20250506,DB2R000100,Postpaid Revenue,20250505,2025-05-06,"85,777,928","171,555,856","171,418,271","171,418,271",0,"3,006,332",0,0,4,170,183
5,20250506,DB2R000100,Postpaid Revenue,20250506,2025-05-06,"89,539,376","179,078,752","178,925,725","178,925,725",0,"3,191,842",0,0,4,170,185


In [11]:
''' DB2R010100 : Postpaid Revenue B2C '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20250501


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250506,DB2R010100,Postpaid Revenue B2C,20250501,2025-05-06,"6,181,046","12,362,093","12,351,158","12,351,158",0,"302,660",0,0,4,170,183
1,20250506,DB2R010100,Postpaid Revenue B2C,20250502,2025-05-06,"95,575,342","191,150,684","190,964,032","190,964,032",0,"3,690,055",0,0,4,170,183
2,20250506,DB2R010100,Postpaid Revenue B2C,20250503,2025-05-06,"87,122,207","174,244,415","174,104,690","174,104,690",0,"3,167,446",0,0,4,170,183
3,20250506,DB2R010100,Postpaid Revenue B2C,20250504,2025-05-06,"87,539,841","175,079,683","174,941,532","174,941,532",0,"3,111,395",0,0,4,170,183
4,20250506,DB2R010100,Postpaid Revenue B2C,20250505,2025-05-06,"84,447,712","168,895,424","168,759,424","168,759,424",0,"3,002,337",0,0,4,170,183
5,20250506,DB2R010100,Postpaid Revenue B2C,20250506,2025-05-06,"87,253,696","174,507,391","174,354,169","174,354,169",0,"3,188,110",0,0,4,170,185


In [12]:
''' DB2R020100 : Postpaid Revenue B2B '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20250501


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250506,DB2R020100,Postpaid Revenue B2B,20250501,2025-05-06,"3,776,518","7,553,037","7,552,875","7,552,875",0,"11,275",0,0,4,170,183
1,20250506,DB2R020100,Postpaid Revenue B2B,20250502,2025-05-06,"135,885,687","271,771,374","271,240,377","271,240,377",0,"5,098,282",0,0,4,170,183
2,20250506,DB2R020100,Postpaid Revenue B2B,20250503,2025-05-06,"3,203,657","6,407,315","6,407,209","6,407,209",0,"9,471",0,0,4,169,182
3,20250506,DB2R020100,Postpaid Revenue B2B,20250504,2025-05-06,"860,656","1,721,313","1,720,482","1,720,482",0,"4,422",0,0,4,167,180
4,20250506,DB2R020100,Postpaid Revenue B2B,20250505,2025-05-06,"1,330,216","2,660,432","2,658,848","2,658,848",0,"3,995",0,0,4,170,183
5,20250506,DB2R020100,Postpaid Revenue B2B,20250506,2025-05-06,"2,285,680","4,571,360","4,571,556","4,571,556",0,"3,731",0,0,4,170,183


In [13]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]